In [1]:
import sys
import numpy as np
import os
import pandas as pd
# sys.path.insert(0,'/global/homes/b/bpb/metaiq/')
sys.path.insert(0,'/global/homes/b/bpb/repos/pactolus/')
import score_frag_dag as sfg

MPI not available. Running in serial.


/global/common/software/m2650/python-cori/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# path = '/scratch2/scratchdirs/bpb/level_3_trees/'
# path = '/project/projectdirs/metatlas/projects/clean_pactolus_trees/'
path = '/projectb/sandbox/metatlas/projects/clean_pactolus_trees/'

In [3]:
# path = '/project/projectdirs/openmsi/projects/ben_trees/'
sfg = reload(sfg)
tree_file = sfg.make_file_lookup_table_by_MS1_mass(path=path)
np.save(os.path.join(path,'tree_lookup.npy'),tree_file)

In [ ]:
temp = os.path.join(os.path.join(path,'tree_lookup.npy'))
%ls -lt $temp

In [ ]:
import numpy as np
import h5py as h5py

trees = np.load('/projectb/sandbox/metatlas/projects/clean_pactolus_trees/tree_lookup.npy')

counter = 0
bad_files = []
for f in trees['filename']:
    with h5py.File(f,'r+') as file_reader:
        for inchi_key in file_reader.keys():
            for depth_key in file_reader[inchi_key].keys():
                if not 'max_depth=' in depth_key:
                    counter += 1
                    bad_files.append(f)
#                     print(f)
#                     print(inchi_key,depth_key)
#                     tree = file_reader[inchi_key][depth_key][:]
#                     print('')

print(counter)

with open('/global/homes/b/bpb/Downloads/bad_files.txt','w') as fid:
    fid.write('\n'.join(bad_files))

In [ ]:
tree[-1]

In [ ]:
filename = tree_file[-2][0]
with sfg.h5py.File(filename,'r') as file_reader:
    group_key = file_reader.keys()[0]
    data_key = file_reader[group_key].keys()[0]
    tree = file_reader[group_key][data_key][:]


In [ ]:
t2 = np.load(os.path.join(path,'tree_lookup.npy'))
df = pd.DataFrame(t2)

In [ ]:
df.shape

In [ ]:
df['inchi_key'] = df.filename.apply(lambda x: x.split('_')[-1].replace('.h5',''))

In [ ]:
missing_ema_idx = []
ema_df = pd.read_pickle('/project/projectdirs/metatlas/projects/magi_paper/ema_pos_hilic_atlas_50447.pkl')
missing_inchi_key = []
for i,row in ema_df.iterrows():
    if df[df.inchi_key == row.inchi_key].shape[0] == 0:
        missing_ema_idx.append(i)
        missing_inchi_key.append(row.inchi_key)
        
print len(missing_ema_idx)
ema_df = pd.read_pickle('/project/projectdirs/metatlas/projects/magi_paper/ema_neg_hilic_atlas_50447.pkl')

for i,row in ema_df.iterrows():
    if df[df.inchi_key == row.inchi_key].shape[0] == 0:
        if not row.inchi_key in missing_inchi:
            missing_ema_idx.append(i)
            missing_inchi_key.append(row.inchi_key)
        
print len(missing_ema_idx)


In [ ]:
sys.path.insert(0,'/global/homes/b/bpb/metatlas/')
from metatlas.compounds import structure_cleaning as mol_clean
from rdkit import Chem

In [ ]:
delete_list = []
# delete_list.append(df[df.ms1_mass == -1]['filename'].tolist())
# print(len(delete_list[0]))
# trim_df = df[df['ms1_mass']>0].copy()

# delete_list.append(trim_df[~trim_df.isin(df_dedup).all(1)]['filename'].tolist())
# print(len(delete_list[0])),(len(delete_list[1]))
# delete_list = [aa for a in delete_list for aa in a]
# len(delete_list)

for i,row in df.iterrows():
    if row.inchi and row.ms1_mass > 100:
        mol = Chem.MolFromInchi(row.inchi)
        try:
            (new_mol,is_neutral) = mol_clean.NeutraliseCharges(mol)
            if row.inchi != Chem.MolToInchi(new_mol):
                delete_list.append(i)
        except:
            delete_list.append(i)
            print row.inchi
    else:
        delete_list.append(i)



In [ ]:
df_junk = df.loc[delete_list]
trim_df = df.copy()
trim_df = trim_df[~df.isin(df_junk).all(1)]

In [ ]:
trim_df.shape

In [ ]:
trim_df.sort_values(['max_depth'],ascending=False,inplace=True)
df_dedup = trim_df.drop_duplicates(subset=['inchi_key'])
df_dedup.shape

In [ ]:
# indir = '/project/projectdirs/metatlas/projects/pactolus_trees/'
# outdir = '/project/projectdirs/metatlas/projects/clean_pactolus_trees/'
# keep_files = []
# for i,row in df_dedup.iterrows():
#     keep_files.append(os.path.basename(row.filename))


In [ ]:
# out_str = 'cp %s{%s} %s'%(indir,','.join(keep_files),outdir)
# with open('/global/u2/b/bpb/copy_files.sh','w') as fid:
#     fid.write(out_str)

In [ ]:
missing_ema_idx = []
ema_df = pd.read_pickle('/project/projectdirs/metatlas/projects/magi_paper/ema_pos_hilic_atlas_50447.pkl')
missing_inchi_key = []
for i,row in ema_df.iterrows():
    if df_dedup[df_dedup.inchi_key == row.inchi_key].shape[0] == 0:
        missing_ema_idx.append(i)
        missing_inchi_key.append(row.inchi_key)
        
print len(missing_ema_idx)
ema_df = pd.read_pickle('/project/projectdirs/metatlas/projects/magi_paper/ema_neg_hilic_atlas_50447.pkl')

for i,row in ema_df.iterrows():
    if df_dedup[df_dedup.inchi_key == row.inchi_key].shape[0] == 0:
        if not row.inchi_key in missing_inchi:
            missing_ema_idx.append(i)
            missing_inchi_key.append(row.inchi_key)
        
print len(missing_ema_idx)


In [ ]:
from metatlas import metatlas_objects as metob

In [ ]:
# inchis = []
# for ik in missing_inchi_key:
#     cpds = metob.retrieve('Compounds',inchi_key = ik,username='*')[-1]
#     print cpds.name
#     inchis.append(cpds.inchi)

In [ ]:
# pd.unique(inchis)

In [ ]:
# with open('/global/u2/b/bpb/Downloads/missing_ema_cpds.txt','w') as fid:
#     fid.write('\n'.join(pd.unique(inchis)))

In [ ]:
from matplotlib import pyplot as plt
%matplotlib notebook
fig = plt.figure()
plt.hist(df_dedup.ms1_mass,bins = 50,range=(0,1200))
plt.xlabel('molecular weight')
plt.ylabel('# trees')
plt.show()

In [ ]:
from matplotlib import pyplot as plt
%matplotlib notebook
fig = plt.figure()
plt.hist(df[df.ms1_mass > 0].ms1_mass,bins = 50,range=(0,1200))
plt.xlabel('molecular weight')
plt.ylabel('# trees')
plt.show()

In [ ]:
fig = plt.figure()
out = plt.hist(np.log10(df[df.time_to_build > 0].time_to_build/60),bins = 100)#,range=(0,1200))
plt.xlabel('LOG10 time to build (minutes)')
plt.ylabel('# trees')

plt.show()

In [ ]:
fig = plt.figure()

out = plt.plot(df.ms1_mass,df.time_to_build/60,'.')#,range=(0,1200))
plt.ylabel('depth of 3\ntime to build tree (min)')
ax = plt.gca()
ax.set_yscale('log')
plt.xlabel('molecular weight')
plt.xlim(0,1200)
plt.show()

### each row is a compound structure
* inchi_key
* is it in metatlas
* is a tree made to a depth of 5
* inchi_key has exact chembl preferred name & synonyms
* inchi_key has exact pubchem synonyms
* partial inchi_key has pubchem synonyms
* is a job submitted to build the tree


## fields to add to metatlas db
* path to tree file
* prefered name from chembl
* full inchikey synonyms from chembl
* full inchikey synonyms from pubchem
* partial inchikey synonyms from pubchem
* partial inchikey pubchem cid
* inchikey pubchem cid

## submit inchi list to nersc
* trees that are not built need to be built
* small trees are fast
* large trees take 10 minutes to days
* once they are built register their location in the database
* some structures can't be built for various reasons

## stop-gap plan
* add location of tree to database
* all trees must be in database
* not all database compound have trees

## store complete fragmentation of trees
* store all unique fragments
* store parent child relationship for each fragment
* at some point generating fragment is faster than storing
* only store immediate relationship between fragment
* each fragment has an id
* store a table linking parent and child